### 기본 CF 알고리즘

> 모든 사용자 간의 평가 유사도를 측정

> 현재 추천 대상이 되는 사람과 다른 사용자의 유사도 추출

> 현재 사용자가 평가하지 않은 모든 아이템에 대해 현재 사용자의 예상 평가 값을 구한다

> 아이템 중 예상 평가 값이 가장 높은 N개의 아이템을 추천한다

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/content/drive/MyDrive/recosys/u.user', sep = '|', names = u_cols, encoding = 'latin-1')

i_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDB URL', 'unknown', 'Action',
          'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Horror',
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'war', 'western']

movies = pd.read_csv('/content/drive/MyDrive/recosys/u.item', sep = '|', names = i_cols, encoding = 'latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/recosys/u.data', sep = '\t', names = r_cols, encoding = 'latin-1')

In [6]:
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [3]:
rating_matrix = x_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

In [7]:
def CF_simple(user_id, movie_id):
  if movie_id in rating_matrix:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()

  else:
    mean_rating = 3.0

  return mean_rating

print(score(CF_simple))

1.0133343516227022


### 이웃을 고려한 CF

> 사용자 중 유사도가 높은 사람을 선정해서 그 사람들의 평점만 가지고 예측

> KNN 방법 사용

In [8]:
def score(model, neighbor_size = 0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

def cf_knn(user_id, movie_id, neighbor_size = 0):
  if movie_id in rating_matrix:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index

    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()

    else:
      if len(sim_scores) > 1:   #이 조건은 평가자 수가 최소 2명 이상인 경우에만 그 다음 계산을 진행하도록 위함이다.
        neighbor_size = min(neighbor_size, len(sim_scores))    #지정된 neighbor_size과 해당 영화를 평가한 총 사용자 수 중 적은 수를 이웃으로 설정
        sim_scores = np.array(sim_scores)    #argsort를 사용하기 위함
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]    #유사도 가장 높은 k명의 사용자 설정
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]    #k명 사용자에 대한 평점 추출
        mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()

      else:
        mean_rating = 3.0

  else:
    mean_rating = 3.0

  return mean_rating

score(cf_knn, neighbor_size = 30)

1.007359306227529

In [10]:
#### 위 코드는 동일
rating_matrix = ratings.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')   #실제 추천을 할 때는 train data가 아닌 전체 데이터 사용

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

def recommender(user, n_items = 10, neighbor_size =20):
  predictions = []
  rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    #현 사용자가 이미 평가한 영화 표시
  items = rating_matrix.loc[user].drop(rated_index)   #이미 평가한 영화는 제외

  for item in items.index:
    predictions.append(cf_knn(user, item, neighbor_size))   #각 아이템에 대해 cf_knn 함수를 통해 구한 예상 평점을 predictions에 추가

  recommendations = pd.Series(data = predictions, index = items.index, dtype = float)
  recommendations = recommendations.sort_values(ascending = False)[:n_items]
  recommended_items = movies.loc[recommendations.index]['title']    #추천할 영화의 id에 해당하는 제목 추출
  return recommended_items

recommender(user = 2, n_items = 5, neighbor_size = 30)    #2번 사용자에게 5개 영화 추천, 이웃 크기는 30

movie_id
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
1467                                     Cure, The (1995)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

### 최적의 이웃 크기 찾기

In [11]:
rating_matrix = x_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

for neighbor_size in range(10, 61, 10):
  print("Neighbor size = %d: RMSE = %4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10: RMSE = 1.026028
Neighbor size = 20: RMSE = 1.013168
Neighbor size = 30: RMSE = 1.011034
Neighbor size = 40: RMSE = 1.010491
Neighbor size = 50: RMSE = 1.010350
Neighbor size = 60: RMSE = 1.010648


### 사용자 기반 CF

In [12]:
### 앞 코드는 동일
rating_mean = rating_matrix.mean(axis = 1)
rating_bias = (rating_matrix.T - rating_mean).T

def cf_knn_bias(user_id, movie_id, neighbor_size = 0):
  if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index

    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
      prediction = prediction + rating_mean[user_id]

    else:
      if len(sim_scores) > 1:   #이 조건은 평가자 수가 최소 2명 이상인 경우에만 그 다음 계산을 진행하도록 위함이다.
        neighbor_size = min(neighbor_size, len(sim_scores))    #지정된 neighbor_size과 해당 영화를 평가한 총 사용자 수 중 적은 수를 이웃으로 설정
        sim_scores = np.array(sim_scores)    #argsort를 사용하기 위함
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]    #유사도 가장 높은 k명의 사용자 설정
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]    #k명 사용자에 대한 평점 추출
        prediction = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
        prediction = prediction + rating_mean[user_id]

      else:
        prediction = rating_mean[user_id]

  else:
    prediction = rating_mean[user_id]

  return prediction

score(cf_knn_bias, 30)

0.9479610425363816